## 영화 리뷰 데이터 분류하기 (NSMC 데이터 학습하기)

In [39]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

from konlpy.tag import Okt
okt = Okt()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SH501\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
def load_data(file_path):
    train = []
    count = 0
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            if count == 500: break
            line = line.strip()
            id, doc, label = line.split('\t')
            if label == '1': label = 'pos'
            elif label == '0': label = 'neg'
            train.append((doc, label))
            count += 1
                    
    return train

In [41]:
def pos_tokenize(raw_sent):
    pos_sent = []
    sent = okt.pos(raw_sent, norm=True, stem=True)
    
    for tup in sent:
        word, tag = tup[0], tup[1]
        word_tag = word + '/' + tag
        pos_sent.append(word_tag)
    return ' '.join(pos_sent)

In [42]:
def make_word_dict(train, use_morph=False):
    all_words = set()
    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph: sent = pos_tokenize(sent)
        words = word_tokenize(sent)
        for word in words:
            all_words.add(word)
    return all_words

In [43]:
def make_train_feats(train, all_words, use_morph=False):
    train_features = []

    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph: sent = pos_tokenize(sent)
        words = word_tokenize(sent)   
        tmp = {set_word: (set_word in words) for set_word in all_words}

        sent_tup = (tmp, label)
        train_features.append(sent_tup)

    for i in range(len(train_features)):
        print(train_features[i])
    return train_features

In [44]:
train = load_data('./nsmc/ratings_train.txt')
train = train[1:]
print(train[:5])

[('아 더빙.. 진짜 짜증나네요 목소리', 'neg'), ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'pos'), ('너무재밓었다그래서보는것을추천한다', 'neg'), ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'neg'), ('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'pos')]


In [45]:
use_morph = True
all_words = make_word_dict(train, use_morph)
print('단어 집합 개수: ', len(all_words))

train_features = make_train_feats(train, all_words, use_morph)

단어 집합 개수:  2322


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [48]:
classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features(n=15)

Most Informative Features
                       ; = True              neg : pos    =      8.2 : 1.0
          재미없다/Adjective = True              neg : pos    =      8.2 : 1.0
                주인공/Noun = True              neg : pos    =      7.5 : 1.0
                 최고/Noun = True              pos : neg    =      6.9 : 1.0
                  뭐/Noun = True              neg : pos    =      6.8 : 1.0
           재밌다/Adjective = True              pos : neg    =      6.5 : 1.0
                 내용/Noun = True              neg : pos    =      6.2 : 1.0
       ㅡㅡ/KoreanParticle = True              neg : pos    =      6.1 : 1.0
                스토리/Noun = True              neg : pos    =      6.1 : 1.0
                 다시/Noun = True              pos : neg    =      5.9 : 1.0
                 인생/Noun = True              pos : neg    =      5.3 : 1.0
            못/VerbPrefix = True              neg : pos    =      4.9 : 1.0
                마지막/Noun = True              pos : neg    =      4.7 : 1.0

# 테스트해보기

In [47]:
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/" +
                           "ratings_test.txt", "ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x21c96a09180>)

In [56]:
test = load_data("ratings_test.txt") # '영화가 재미없다 ㅡㅡ'
test = test[1:]
test_sent = test[0][0]

test_sent = pos_tokenize(test_sent)
print(test_sent)
words = word_tokenize(test_sent)
test_feature = {set_word: (set_word in words) for set_word in all_words}

classifier.classify(test_feature)

굳다/Adjective ㅋ/KoreanParticle


'pos'